In [4]:
import os
import json
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.manifold import TSNE
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

In [5]:
def detect_id_column(df: pd.DataFrame) -> str:
    """Detecta una columna identificadora (país/nombre). Si no hay, crea 'Country'."""
    candidates = ["pais", "País", "PAIS", "country", "Country", "COUNTRY", "Pais"]
    for c in candidates:
        if c in df.columns:
            return c
    # si no encuentra, usa la primera columna de texto razonable
    for c in df.columns:
        if df[c].dtype == "object":
            return c
    # si no hay, crea una
    df.insert(0, "Country", [f"Country_{i}" for i in range(len(df))])
    return "Country"


def safe_silhouette(X: np.ndarray, labels: np.ndarray) -> float:
    """Calcula silhouette si es válido, si no retorna np.nan."""
    unique = set(labels)
    # si todos son un mismo cluster o todos ruido, no aplica
    if len(unique) <= 1:
        return np.nan
    # si todos son clusters distintos tampoco aplica (silhouette requiere k>=2 y <= n-1)
    if len(unique) >= len(labels):
        return np.nan
    try:
        return silhouette_score(X, labels)
    except Exception:
        return np.nan

In [6]:
def main():
    # === 0) CONFIG ===
    csv_path = "dataset_generos_musicales.csv"
    out_dir = "./m7_output"
    zip_path = "./entrega_evaluacion_M7.zip"
    os.makedirs(out_dir, exist_ok=True)
    RNG = 42

    # === 1) Cargar dataset y preparar identificador ===
    df = pd.read_csv(csv_path)
    id_col = detect_id_column(df)
    df = df[[id_col] + [c for c in df.columns if c != id_col]]

    # === 2) Seleccionar numéricas y completar NaN con la media ===
    num_df = df.select_dtypes(include=[np.number]).copy()
    num_df = num_df.fillna(num_df.mean(numeric_only=True))

    # Proteger si no hay columnas numéricas
    if num_df.shape[1] == 0:
        raise ValueError("El dataset no contiene columnas numéricas para analizar.")

    # === 3) Escalamiento ===
    X = StandardScaler().fit_transform(num_df.values)
    n = len(df)

    # === 4) K-Means: método del codo + silhouette ===
    k_values = list(range(2, min(10, n) + 1))
    inertias, silhouettes = [], []
    best_k, best_sil = None, -1.0

    for k in k_values:
        km = KMeans(n_clusters=k, n_init=25, random_state=RNG)
        labels_k = km.fit_predict(X)
        inertias.append(km.inertia_)
        sil_k = safe_silhouette(X, labels_k)
        silhouettes.append(sil_k)
        if not np.isnan(sil_k) and sil_k > best_sil:
            best_sil, best_k = sil_k, k

    k_opt = best_k if best_k is not None else 3
    kmeans_final = KMeans(n_clusters=k_opt, n_init=50, random_state=RNG)
    kmeans_labels = kmeans_final.fit_predict(X)

    # === 5) Clustering Jerárquico (Ward) ===
    Z = linkage(X, method="ward")
    hier_labels = fcluster(Z, t=k_opt, criterion="maxclust") - 1  # 0-based
    sil_hier = safe_silhouette(X, hier_labels)

    # === 6) DBSCAN: rejilla simple (rápida) ===
    eps_candidates = [0.3, 0.5, 0.7, 1.0]
    minpts_candidates = sorted(set([2, 3, max(2, int(np.log(max(n, 2))))]))
    best_db_sil = -1.0
    best_eps = np.nan
    best_minpts = np.nan
    db_labels = np.array([-1] * n)

    for eps in eps_candidates:
        for m in minpts_candidates:
            if m >= n:
                continue
            db = DBSCAN(eps=eps, min_samples=m, metric="euclidean")
            labels_db = db.fit_predict(X)
            # silhouette solo con puntos no-ruido y >=2 clusters
            non_noise = labels_db != -1
            n_clusters = len(set(labels_db)) - (1 if -1 in labels_db else 0)
            if n_clusters >= 2 and non_noise.any():
                sil_db = safe_silhouette(X[non_noise], labels_db[non_noise])
            else:
                sil_db = np.nan
            if not np.isnan(sil_db) and sil_db > best_db_sil:
                best_db_sil = sil_db
                best_eps, best_minpts, db_labels = eps, m, labels_db

    # === 7) PCA ===
    pca_full = PCA(random_state=RNG)
    pca_full.fit(X)
    explained = pca_full.explained_variance_ratio_
    cum_explained = np.cumsum(explained)
    n_components_90 = int(np.searchsorted(cum_explained, 0.90) + 1)
    pca2 = PCA(n_components=2, random_state=RNG)
    X_pca2 = pca2.fit_transform(X)

    # === 8) t-SNE (opcional: se ejecuta si hay suficientes muestras) ===
    X_tsne = None
    tsne_name = None
    if n >= 5:
        perplexity = max(5, min(30, n // 3))
        if perplexity < n:
            tsne = TSNE(n_components=2, perplexity=perplexity, init="pca",
                        learning_rate="auto", n_iter=750, random_state=RNG)
            X_tsne = tsne.fit_transform(X)

    # === 9) Guardar salidas ===

    # 9.1 EDA breve
    eda = {
        "shape": df.shape,
        "id_column": id_col,
        "numeric_columns": num_df.columns.tolist()
    }
    with open(os.path.join(out_dir, "eda_summary.json"), "w", encoding="utf-8") as f:
        json.dump(eda, f, ensure_ascii=False, indent=2)

    # 9.2 Gráficos (un gráfico por figura; sin estilos ni colores personalizados)

    # Codo (Inertia vs K)
    plt.figure()
    plt.plot(k_values, inertias, marker="o")
    plt.title("Método del codo (Inertia vs K)")
    plt.xlabel("K")
    plt.ylabel("Inertia")
    plt.tight_layout()
    plt.savefig(os.path.join(out_dir, "elbow_inertia.png"), dpi=200)
    plt.close()

    # Silhouette vs K
    plt.figure()
    plt.plot(k_values, silhouettes, marker="o")
    plt.title("Coeficiente de Silueta vs K")
    plt.xlabel("K")
    plt.ylabel("Silhouette")
    plt.tight_layout()
    plt.savefig(os.path.join(out_dir, "silhouette_vs_k.png"), dpi=200)
    plt.close()

    # Dendrograma (Ward)
    plt.figure(figsize=(9, 5))
    dendrogram(Z, labels=df[id_col].astype(str).values, orientation="top", leaf_rotation=90)
    plt.title("Dendrograma (Ward)")
    plt.xlabel(id_col)
    plt.ylabel("Distancia")
    plt.tight_layout()
    plt.savefig(os.path.join(out_dir, "dendrograma.png"), dpi=200)
    plt.close()

    # PCA: Varianza acumulada
    plt.figure()
    plt.plot(range(1, len(cum_explained) + 1), cum_explained, marker="o")
    plt.axhline(0.90)
    plt.title("PCA - Varianza acumulada")
    plt.xlabel("Componentes")
    plt.ylabel("Varianza acumulada")
    plt.tight_layout()
    plt.savefig(os.path.join(out_dir, "pca_varianza_acumulada.png"), dpi=200)
    plt.close()

    # PCA: Scatter 2D con anotaciones (sin especificar colores)
    plt.figure()
    plt.scatter(X_pca2[:, 0], X_pca2[:, 1])
    for i, name in enumerate(df[id_col].astype(str).values):
        plt.annotate(name, (X_pca2[i, 0], X_pca2[i, 1]))
    plt.title(f"PCA - Proyección 2D (K-Means K={k_opt})")
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.tight_layout()
    plt.savefig(os.path.join(out_dir, "pca_scatter_2d.png"), dpi=200)
    plt.close()

    # t-SNE: Scatter 2D con anotaciones (si corrió)
    if X_tsne is not None:
        plt.figure()
        plt.scatter(X_tsne[:, 0], X_tsne[:, 1])
        for i, name in enumerate(df[id_col].astype(str).values):
            plt.annotate(name, (X_tsne[i, 0], X_tsne[i, 1]))
        plt.title(f"t-SNE 2D (perplexity={perplexity})")
        plt.xlabel("Dim 1")
        plt.ylabel("Dim 2")
        plt.tight_layout()
        tsne_name = "tsne_perplexity_{:d}.png".format(perplexity)
        plt.savefig(os.path.join(out_dir, tsne_name), dpi=200)
        plt.close()

    # 9.3 Asignaciones de clusters
    result_df = df[[id_col]].copy()
    result_df["kmeans_k"] = k_opt
    result_df["kmeans_label"] = kmeans_labels
    result_df["hier_label(k_opt)"] = hier_labels
    result_df["dbscan_eps"] = best_eps
    result_df["dbscan_min_samples"] = best_minpts
    result_df["dbscan_label"] = db_labels
    result_df.to_csv(os.path.join(out_dir, "cluster_assignments.csv"), index=False, encoding="utf-8")

    # 9.4 Resumen/conclusiones automáticas
    lines = []
    lines.append("=== RESUMEN DE RESULTADOS ===")
    lines.append(f"ID: '{id_col}'. Muestras={n}; Vars numéricas={num_df.shape[1]}.")
    lines.append("")
    lines.append("K-Means:")
    lines.append(f"- K evaluados: {k_values}")
    lines.append(f"- K óptimo por silueta: {k_opt} (silhouette={best_sil:.3f})")
    lines.append("")
    lines.append("Clustering Jerárquico (Ward):")
    if not np.isnan(sil_hier):
        lines.append(f"- Silhouette (con {k_opt} clusters): {sil_hier:.3f}")
    else:
        lines.append("- Silhouette no válida para este particionado")
    lines.append("")
    lines.append("DBSCAN:")
    if not np.isnan(best_eps):
        n_clusters_db = len(set(db_labels)) - (1 if -1 in db_labels else 0)
        lines.append(f"- Mejor configuración: eps={best_eps}, min_samples={best_minpts}, clusters={n_clusters_db}")
        if best_db_sil > -0.5:
            lines.append(f"- Silhouette (sin ruido) ≈ {best_db_sil:.3f}")
    else:
        lines.append("- No se encontró configuración útil de DBSCAN para separar >1 clúster.")
    lines.append("")
    lines.append("PCA:")
    lines.append(f"- Componentes para ≥90% varianza: {n_components_90}")
    lines.append(f"- Varianza explicada acumulada (primeras 5): {np.round(cum_explained[:5], 3).tolist()}")
    lines.append("")
    if X_tsne is not None:
        lines.append(f"t-SNE: generado con perplexity={perplexity} (n_iter=750)")
    else:
        lines.append("t-SNE: omitido (muestras insuficientes para perplexity estándar).")
    lines.append("")
    lines.append("Interpretación:")
    lines.append("- K-Means funciona bien con clusters esféricos; Ward revela estructura jerárquica; DBSCAN detecta densidades irregulares y ruido.")

    summary_text = "\n".join(lines)
    with open(os.path.join(out_dir, "resumen_resultados.txt"), "w", encoding="utf-8") as f:
        f.write(summary_text)

    # 9.5 Crear ZIP con todo
    with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as z:
        for fname in [
            "eda_summary.json",
            "elbow_inertia.png",
            "silhouette_vs_k.png",
            "dendrograma.png",
            "pca_varianza_acumulada.png",
            "pca_scatter_2d.png",
            "cluster_assignments.csv",
            "resumen_resultados.txt",
        ]:
            z.write(os.path.join(out_dir, fname), arcname=fname)
        if tsne_name:
            z.write(os.path.join(out_dir, tsne_name), arcname=tsne_name)

    print("Listo ✅")
    print(f"- Carpeta de salida: {os.path.abspath(out_dir)}")
    print(f"- ZIP de entrega:   {os.path.abspath(zip_path)}")
    print("\nResumen rápido:\n")
    print(summary_text)


if __name__ == "__main__":
    main()


/usr/local/lib/python3.12/dist-packages/sklearn/manifold/_t_sne.py:1164: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(


Listo ✅
- Carpeta de salida: /content/m7_output
- ZIP de entrega:   /content/entrega_evaluacion_M7.zip

Resumen rápido:

=== RESUMEN DE RESULTADOS ===
ID: 'País'. Muestras=8; Vars numéricas=8.

K-Means:
- K evaluados: [2, 3, 4, 5, 6, 7, 8]
- K óptimo por silueta: 5 (silhouette=0.283)

Clustering Jerárquico (Ward):
- Silhouette (con 5 clusters): 0.283

DBSCAN:
- No se encontró configuración útil de DBSCAN para separar >1 clúster.

PCA:
- Componentes para ≥90% varianza: 4
- Varianza explicada acumulada (primeras 5): [0.46, 0.698, 0.871, 0.957, 0.99]

t-SNE: generado con perplexity=5 (n_iter=750)

Interpretación:
- K-Means funciona bien con clusters esféricos; Ward revela estructura jerárquica; DBSCAN detecta densidades irregulares y ruido.


Comparación de métodos

K-Means vs. Jerárquico (Ward) vs. DBSCAN

K-Means: asume grupos “más o menos esféricos” y de tamaño similar. Con tus datos arrojó K=5* con silhouette = 0,283.

Jerárquico (Ward): no fija K de entrada (se corta el dendrograma a un nivel), optimiza varianza intragrupo. Con 5 clústeres dio la misma silhouette = 0,283.

DBSCAN: busca “islas” densas y marca ruido. No encontró una configuración que separara >1 clúster, señal de que no hay núcleos densos claros en 8 observaciones.

Cuál funcionó mejor y por qué

Empate técnico entre K-Means y Ward (idéntico 0,283).
En la práctica:

K-Means es más simple de reportar y reproducir (etiquetas estables, fácil de aplicar a nuevos datos).

Ward aporta el dendrograma, útil para ver subestructura y umbral de corte.

DBSCAN no es adecuado aquí (muestra pequeña, sin densidades claras).

PCA vs. t-SNE (visualización)

PCA: con 2 componentes acumulas ~70% de varianza; con 4 llegas a >90%.

Ventaja: muestra relaciones globales (gradientes/continuos) y ejes interpretables.

t-SNE (perplexity=5): enfatiza vecindades locales; en muestras pequeñas tiende a separar “islas” más de lo que realmente están en el espacio original.

Ventaja: buena para ver microgrupos; desventaja: distorsiona distancias globales y no es interpretable por ejes.

¿Cuál permitió visualizar mejor la relación entre países?

Si por “relación” entendemos estructura global y comparabilidad entre todos, PCA gana.

Si buscas agrupar cercanías locales (quién se parece con quién), t-SNE puede verse más “limpio”, pero con cautela por la muestra pequeña.
En resumen: PCA para panorama general, t-SNE para micro-clusters.

Interpretación

¿Los clústeres reflejan similitudes culturales o geográficas?

Con solo 8 países y una separación moderada (silhouette 0,283) y 5 clústeres (muchos grupos pequeños), la evidencia de bloques culturales/geográficos fuertes es débil.

El hecho de que DBSCAN no encuentre núcleos densos refuerza que las diferencias no son muy marcadas ni hay “sub-escenas” muy compactas.

Probable lectura: hay afinidades puntuales (pares o tríos que se parecen) más que grandes constelaciones regionales.

Relación con tendencias globales de consumo musical

El streaming global y los algoritmos de recomendación tienden a homogeneizar gustos (pop internacional, colaboraciones cross-border), lo que reduce la formación de bloques regionales muy nítidos.

Aun así, nichos locales/globales (p. ej., escenas urbanas latinas, K-pop, afrobeats, regional mexicano, corridos, etc.) pueden generar pequeñas “islas” de similitud—coherente con varios micro-clusters y silhouette moderada.

En suma: la foto sugiere un equilibrio entre globalización del gusto (poca densidad fuerte) y particularidades locales (microgrupos), más que clústeres continentales robustos.